In [241]:
import boto3
import numpy as np
from pexpect import pxssh
import time

In [284]:
def instances(ec2,max_count,key_name):
    ec2.create_instances(ImageId='ami-539ac933',InstanceType='t2.micro',MinCount=1,MaxCount=max_count,KeyName=key_name,SecurityGroupIds=['sg-d7ef59b0'])

def get_ip(ec2):
    ips = []
    for instance in ec2.instances.filter(Filters = [
        {
            'Name': 'instance-state-code',
            'Values': ['16']
        }
    ]):
        ips.append(instance.public_ip_address.replace('.','-'))
    return ips

def login(ec2,zone,key_name):
    ips = get_ip(ec2)
    ssh_connections = []
    for ip in ips:
        s = pxssh.pxssh()
        s.login(server='ec2-' + ip + '.' + zone + '.compute.amazonaws.com',username='ubuntu',ssh_key=key_name+'.pem')
        ssh_connections.append(s)
    return ssh_connections

In [247]:
def run_model_on_instance(s,parameters):
    commands = ['git clone https://github.com/account/folder',\
                'mv -v /home/ubuntu/folder/* /home/ubuntu',\
                'rm -rf /home/ubuntu/folder'
               ]
    for command in commands:
        s.sendline(command)
        time.sleep(10)
    s.sendline('bash script_to_install.sh')
    time.sleep(600)
    s.sendline('python script_gp_model.py ' + parameters)
    time.sleep(60)
    #get_results()

In [245]:
def run_model_on_instances(ssh_connections,parameters):
    k = 0
    for ssh_connection in ssh_connections:
        run_model_on_instance(ssh_connection,parameters[k])
        k +=1

In [ ]:
ec2 = boto3.resource('ec2')
key_name = 'name'
instances(ec2,2,key_name)
zone = 'us-west-1'
ssh_connections = login(ec2,zone,key_name)
parameters = ['200 50 0','100 25 1']
run_model_on_instances(ssh_connections,parameters)